# Data Preparation and Quality Assessment

In this notebook I will clean up the raw data and prepare it to be usable for clustering. Additionally, I will implement a comprehensive data quality assessment function to evaluate the dataset's suitability for clustering tasks.

In [53]:
import pandas as pd

df = pd.read_csv("../data/raw/GlobalWeatherRepository.csv")

print(df.head())
print(df.columns)
print(df["condition_text"].unique())

       country     location_name  latitude  longitude        timezone  \
0  Afghanistan             Kabul     34.52      69.18      Asia/Kabul   
1      Albania            Tirana     41.33      19.82   Europe/Tirane   
2      Algeria           Algiers     36.76       3.05  Africa/Algiers   
3      Andorra  Andorra La Vella     42.50       1.52  Europe/Andorra   
4       Angola            Luanda     -8.84      13.23   Africa/Luanda   

   last_updated_epoch      last_updated  temperature_celsius  \
0          1715849100  2024-05-16 13:15                 26.6   
1          1715849100  2024-05-16 10:45                 19.0   
2          1715849100  2024-05-16 09:45                 23.0   
3          1715849100  2024-05-16 10:45                  6.3   
4          1715849100  2024-05-16 09:45                 26.0   

   temperature_fahrenheit condition_text  ...  air_quality_PM2.5  \
0                    79.8  Partly Cloudy  ...                8.4   
1                    66.2  Partly cloudy

In [54]:
df.describe()

,latitude,longitude,last_updated_epoch,temperature_celsius,temperature_fahrenheit,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,...,gust_kph,air_quality_Carbon_Monoxide,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,moon_illumination
count,98214.000000,98214.000000,9.821400e+04,98214.000000,98214.000000,98214.000000,98214.000000,98214.000000,98214.000000,98214.000000,...,98214.000000,98214.000000,98214.000000,98214.000000,98214.000000,98214.000000,98214.000000,98214.000000,98214.000000,98214.000000
mean,19.148049,22.107646,1.737715e+09,22.772745,72.992666,8.232332,13.251953,170.786863,1013.985012,29.942321,...,18.531917,515.785574,62.629229,15.798688,11.233711,26.084961,53.549105,1.758201,2.755361,49.810770
std,24.456192,65.818422,1.262162e+07,8.889949,16.001722,7.861176,12.648939,102.722060,11.355404,0.335272,...,14.650513,842.218264,32.223494,26.012929,40.658920,40.710795,163.611750,0.980364,2.563471,34.985969
min,-41.300000,-175.200000,1.715849e+09,-24.900000,-12.800000,2.200000,3.600000,1.000000,947.000000,27.960000,...,3.600000,-9999.000000,0.000000,0.000000,-9999.000000,0.168000,-1848.150000,1.000000,1.000000,0.000000
25%,3.750000,-6.836100,1.726829e+09,18.100000,64.600000,4.000000,6.500000,83.000000,1010.000000,29.830000,...,10.500000,237.000000,42.000000,1.295000,0.846000,7.400000,10.800000,1.000000,1.000000,15.000000
50%,17.250000,23.316700,1.737716e+09,25.000000,76.900000,6.900000,11.200000,165.000000,1013.000000,29.920000,...,15.800000,327.100000,60.000000,4.995000,2.405000,15.170000,22.015000,1.000000,2.000000,50.000000
75%,40.400000,50.580000,1.748681e+09,28.300000,82.900000,11.400000,18.400000,256.000000,1018.000000,30.050000,...,24.400000,506.900000,79.000000,17.945000,9.065000,29.785000,45.880000,2.000000,3.000000,84.000000
max,64.150000,179.220000,1.759474e+09,49.200000,120.600000,1841.200000,2963.200000,360.000000,3006.000000,88.770000,...,2970.400000,38879.398000,480.700000,427.700000,521.330000,1614.100000,6037.290000,6.000000,10.000000,100.000000


In [55]:
print(df.isnull().sum())

country                         0
location_name                   0
latitude                        0
longitude                       0
timezone                        0
last_updated_epoch              0
last_updated                    0
temperature_celsius             0
temperature_fahrenheit          0
condition_text                  0
wind_mph                        0
wind_kph                        0
wind_degree                     0
wind_direction                  0
pressure_mb                     0
pressure_in                     0
precip_mm                       0
precip_in                       0
humidity                        0
cloud                           0
feels_like_celsius              0
feels_like_fahrenheit           0
visibility_km                   0
visibility_miles                0
uv_index                        0
gust_mph                        0
gust_kph                        0
air_quality_Carbon_Monoxide     0
air_quality_Ozone               0
air_quality_Ni

## Insights on the data

As we see the data is very clean and is well structured. It contains various weather related columns - most of them we won't need. The biggest issue is that the data is over time and we only want to cluster countries. Thus, we need to aggregate the data per country (we will for now just use all data points from 2023 until 2025 - ignoring that this might cause problems with seasonality for now).

The data we aggregate is the following:

- Minimum, Maximum, Mean and Median Temperature from `temperature_celsius`
- Minimum, Maximum, Mean and Median Precipitation from `precip_mm`
- Number of Rainy Days, Heavy Rain Days, Sunny Days and Partly Cloudy Days from `condition_text`
- Total Number of Days with measurements from `last_updated`

Those features I chose because they are most relevant for the growth of forests and therefore for the application of carbon sequestration projects. We don't take any geographical data into account to make sure that the clustering is only based on weather data.

In [56]:
df_per_country = (
    df.groupby("country")
    .agg(
        temperature_min=("temperature_celsius", "min"),
        temperature_max=("temperature_celsius", "max"),
        temperature_mean=("temperature_celsius", "mean"),
        temperature_median=("temperature_celsius", "median"),
        precip_mm_mean=("precip_mm", "mean"),
        precip_mm_median=("precip_mm", "median"),
        number_of_days_with_precip_mm_gt_0=("precip_mm", lambda x: (x > 0).sum()),
        rain_days=(
            "condition_text",
            lambda x: x.str.lower().str.contains("rain").sum(),
        ),
        heavy_rain_days=(
            "condition_text",
            lambda x: x.str.lower().str.contains("heavy rain").sum(),
        ),
        sun_days=("condition_text", lambda x: x.str.lower().str.contains("sun").sum()),
        partly_cloudy_days=(
            "condition_text",
            lambda x: x.str.lower().str.contains("partly cloudy").sum(),
        ),
        number_of_datapoints=("last_updated", "count"),
    )
    .reset_index()
)
df_per_country.describe()

,temperature_min,temperature_max,temperature_mean,temperature_median,precip_mm_mean,precip_mm_median,number_of_days_with_precip_mm_gt_0,rain_days,heavy_rain_days,sun_days,partly_cloudy_days,number_of_datapoints
count,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000,211.000000
mean,11.652133,33.571564,23.231985,23.750711,0.157215,0.042725,156.725118,77.834123,5.502370,144.094787,170.303318,465.469194
std,12.653839,6.341282,6.488185,6.376774,0.238143,0.216047,130.608467,79.491335,8.536406,145.810805,115.406664,203.414245
min,-24.900000,13.500000,6.865278,7.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,29.700000,19.225734,20.100000,0.040371,0.000000,60.000000,25.500000,0.000000,9.500000,78.500000,502.000000
50%,14.400000,33.100000,24.580838,24.950000,0.095604,0.000000,124.000000,61.000000,2.000000,116.000000,177.000000,504.000000
75%,21.950000,37.750000,27.388347,27.625000,0.171561,0.010000,232.000000,103.500000,7.000000,241.500000,246.500000,505.000000
max,45.000000,49.200000,45.000000,45.000000,1.970000,1.970000,773.000000,549.000000,54.000000,826.000000,582.000000,1081.000000


## Drop Countries with Insufficient Data

I recognized, that for some countries, there is only one measurement. This could impact the reliability of the aggregated statistics - that's why I will drop all countries where we have less than 200 measurements.

In [57]:
print(f"Size before cleaning: {len(df_per_country)} rows")

df_per_country_cleaned = df_per_country[
    df_per_country["number_of_datapoints"] >= 200
].reset_index(drop=True)

print(
    f"Size after cleaning: {len(df_per_country_cleaned)} rows. Dropped {len(df_per_country) - len(df_per_country_cleaned)} entries."
)

Size before cleaning: 211 rows
Size after cleaning: 185 rows. Dropped 26 entries.


In [58]:
def comprehensive_quality_assessment(df):
    """Systematic dataset quality evaluation"""

    quality_metrics = {
        "completeness": 1 - df.isnull().sum().sum() / df.size,
        "feature_variation": df.select_dtypes(include=["number"]).std().mean(),
        "sample_adequacy": len(df) > 200,
        # Temporal coverage can be ignored since we aggregated the temporal data
        # 'temporal_span': assess_temporal_coverage(df) if 'date' in df.columns else None
    }

    # Generate quality score
    quality_score = (
        quality_metrics["completeness"] * 0.3
        + min(1, quality_metrics["feature_variation"]) * 0.3
        + int(quality_metrics["sample_adequacy"]) * 0.4
    )

    return quality_metrics, quality_score


quality_metrics, quality_score = comprehensive_quality_assessment(
    df_per_country_cleaned
)

print("Quality Metrics:", quality_metrics)
print("Quality Score:", quality_score)

Quality Metrics: {'completeness': np.float64(1.0), 'feature_variation': np.float64(50.2503541239224), 'sample_adequacy': False}
Quality Score: 0.6


## Data Quality Assessment insights

As we can see, the data is very complete and has a good feature variation. However, the sample adequacy is critically low - this is because we have only 185 countries in the dataset with enough data. Nevertheless, I will proceed with the clustering, since the most relevant countries (Europeans) are included and we can still get interesting insights from the clustering.

In [59]:
df_per_country_cleaned.to_csv("../data/interim/weather_per_country.csv", index=False)